<a href="https://colab.research.google.com/github/Marisagf/ETL_Python/blob/main/ETL_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Projeto 1 - Deletar e Preencher automaticamente os dados ausentes!

In [2]:
# Importanto bibliotecas
import math
import sys, os
import numpy as np
import pandas as pd

In [3]:
# Limitando o nº de colunas
pd.set_option('display.max_columns', 100)

###Importando os Dados

Dataset publico: National Football League (NFL) - 2009 a 2028
Disponível em: https://www.kaggle.com/datasets/maxhorowitz/nflplaybyplay2009to2016

Os arquivos utilizados neste projeto estão disponíveis em:

NFL Play by Play - 2009 - 2017 --> https://www.kaggle.com/code/rtatman/data-cleaning-challenge-handling-missing-values/input?select=NFL+Play+by+Play+2009-2016+%28v3%29.csv

Building_Permits --> https://www.kaggle.com/code/rtatman/data-cleaning-challenge-handling-missing-values/input?select=Building_Permits.csv

In [4]:
# Criamos uma lista para identificar valores ausentes
lista_labels_valores_ausentes = ["n/a", "na", "undefined"]

In [9]:
# Carrega os datasets
# Jogos
dataset = pd.read_csv("/dataset_NFL2009-2017_v4.csv", na_values = lista_labels_valores_ausentes, low_memory=False)
# Licenças de Construção
building_permits = pd.read_csv("/building_permits.csv", na_values = lista_labels_valores_ausentes, low_memory=False)

Atenção: low_memory = False
- Essa opção é necessária, pois a função read_csv() precisa ler o arquivo inteiro para só então determinar o tipo de dado de cada coluna. Como o arquivo de dados da NFL é muito grande, ocorre um erro, se não o fizermos dessa forma. Isso faz com que o Pandas não determine automaticamente os tipos de dados. Você pode ver que isso é verdade, ao buscar as informações do datasetm como mostrado no próximo comando:

In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407688 entries, 0 to 407687
Columns: 102 entries, Date to Season
dtypes: float64(33), int64(31), object(38)
memory usage: 317.3+ MB


In [11]:
dataset.sample(6) # Informa a quantidade de linhas e colunas.

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,yrdline100,ydstogo,ydsnet,GoalToGo,FirstDown,posteam,DefensiveTeam,desc,PlayAttempted,Yards.Gained,sp,Touchdown,ExPointResult,TwoPointConv,DefTwoPoint,Safety,Onsidekick,PuntResult,PlayType,Passer,Passer_ID,PassAttempt,PassOutcome,PassLength,AirYards,YardsAfterCatch,QBHit,PassLocation,InterceptionThrown,Interceptor,Rusher,Rusher_ID,RushAttempt,RunLocation,RunGap,Receiver,Receiver_ID,Reception,ReturnResult,...,Tackler1,Tackler2,FieldGoalResult,FieldGoalDistance,Fumble,RecFumbTeam,RecFumbPlayer,Sack,Challenge.Replay,ChalReplayResult,Accepted.Penalty,PenalizedTeam,PenaltyType,PenalizedPlayer,Penalty.Yards,PosTeamScore,DefTeamScore,ScoreDiff,AbsScoreDiff,HomeTeam,AwayTeam,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,No_Score_Prob,Opp_Field_Goal_Prob,Opp_Safety_Prob,Opp_Touchdown_Prob,Field_Goal_Prob,Safety_Prob,Touchdown_Prob,ExPoint_Prob,TwoPoint_Prob,ExpPts,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
290333,2015-11-01,2015110108,21,4,3.0,13:33,14,813.0,42.0,ARI,26.0,74.0,4,18,0.0,1.0,ARI,CLE,(13:33) (Shotgun) C.Palmer pass short left to ...,1,12,0,0,NaN,NaN,NaN,0,0,NaN,Pass,C.Palmer,00-0021429,1,Complete,Short,4,8,1,left,0,NaN,NaN,NaN,0,NaN,NaN,L.Fitzgerald,00-0022921,1,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,0,NaN,NaN,NaN,0,24.0,20.0,4.0,4.0,CLE,ARI,0,NaN,3,3,3,3,3,0.068755,0.188780,0.005187,0.277200,0.165616,0.003541,0.290920,0.0,0.0,0.023254,2.121995,1.456408,0.665587,0.374910,0.625090,0.288966,0.711034,0.625090,0.085943,0.058668,0.027275,2015
353059,2016-12-11,2016121110,26,5,2.0,12:07,13,-173.0,44.0,NYJ,41.0,59.0,6,12,0.0,1.0,NYJ,SF,(12:07) (Shotgun) B.Powell left end to NYJ 49 ...,1,8,0,0,NaN,NaN,NaN,0,0,NaN,Run,NaN,NaN,0,NaN,NaN,0,0,0,NaN,0,NaN,B.Powell,00-0028064,1,left,end,NaN,NaN,0,NaN,...,N.Bellore,J.Ward,NaN,NaN,0,NaN,NaN,0,0,NaN,0,NaN,NaN,NaN,0,17.0,17.0,0.0,0.0,SF,NYJ,0,NaN,2,2,2,2,2,0.070863,0.111398,0.001409,0.166333,0.266938,0.003479,0.379581,0.0,0.0,1.963494,0.944462,NaN,NaN,0.350003,0.649997,0.261355,0.738645,0.649997,0.088648,NaN,NaN,2016
402893,2017-12-24,2017122404,11,3,3.0,05:48,6,1248.0,39.0,BUF,10.0,10.0,5,59,0.0,0.0,NE,BUF,(5:48) (Shotgun) T.Brady sacked at BUF 16 for ...,1,-6,0,0,NaN,NaN,NaN,0,0,NaN,Sack,NaN,00-0019596,0,NaN,NaN,0,0,1,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,...,K.Williams,NaN,NaN,NaN,0,NaN,NaN,1,0,NaN,0,NaN,NaN,NaN,0,13.0,15.0,-2.0,2.0,NE,BUF,0,NaN,3,3,3,3,3,0.002459,0.029253,0.000053,0.047528,0.569304,0.002939,0.348463,0.0,0.0,3.732471,-1.042183,NaN,NaN,0.537138,0.462862,0.501531,0.498469,0.537138,-0.035607,NaN,NaN,2017
140604,2012-09-23,2012092307,6,1,2.0,01:58,2,2818.0,0.0,BUF,41.0,59.0,6,4,0.0,0.0,BUF,CLE,(1:58) (Shotgun) R.Fitzpatrick pass incomplete...,1,0,0,0,NaN,NaN,NaN,0,0,NaN,Pass,R.Fitzpatrick,00-0023682,1,Incomplete Pass,Deep,17,0,0,middle,0,NaN,NaN,NaN,0,NaN,NaN,St.Johnson,00-0026364,0,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,0,NaN,NaN,NaN,0,14.0,0.0,14.0,14.0,CLE,BUF,0,NaN,2,3,2,3,2,0.023175,0.118785,0.001509,0.179326,0.279411,0.003715,0.394078,0.0,0.0,1.989555,-0.898493,1.542426,-2.440919,0.101806,0.898194,0.114869,0.885131,0.898194,-0.013063,0.021337,-0.034400,2012
397352,2017-12-10,2017121004,11,2,2.0,05:11,6,2111.0,26.0,SF,34.0,66.0,8,31,0.0,1.0,SF,HOU,(5:11) J.Garoppolo pass deep left to K.Juszczy...,1,29,0,0,NaN,NaN,NaN,0,0,NaN,Pass,J.Garoppolo,00-0031345,1,Complete,Deep,26,3,1,left,0,NaN,NaN,NaN,0,NaN,NaN,K.Juszczyk,00-0029892,1,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,NaN,0,NaN,NaN,NaN,0,3.0,3.0,0.0,0.0,HOU,SF,0,NaN,2,3,2,3,2,0.322626,0.097901,0.001809,0.144569,0.177957,0.002555,0.252583,0.0,0.0,0.997762,2.209256,2.104650,0.104606,0.489737,0.510263,0.418948,0.581052,0.510263,0.070789,0.067860,0.002929,2017
297014,2015-11-16,2015111600,9,2,2.0,03:10,4,1990.0,38.0,CIN,26.0,74.0,9,4,0.0,0.0,CIN,HOU,(3:10) (Shotgu

In [12]:
dataset.shape

(407688, 102)

In [13]:
building_permits.sample(6) # Os dados são exibidos em formato tabular com algumas colunas com valores NaN. Esses valores são chamados de valores ausentes (Not a Number).

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,Completed Date,First Construction Document Date,Structural Notification,Number of Existing Stories,Number of Proposed Stories,Voluntary Soft-Story Retrofit,Fire Only Permit,Permit Expiration Date,Estimated Cost,Revised Cost,Existing Use,Existing Units,Proposed Use,Proposed Units,Plansets,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
127510,201606160101,8,otc alterations permit,06/16/2016,0316,018A,445,NaN,Geary,St,0.0,NaN,addition to permit 201503060168 to show all ro...,complete,01/24/2017,06/16/2016,06/20/2016,01/24/2017,06/20/2016,NaN,3.0,3.0,NaN,NaN,06/15/2017,1.0,1.0,theater,0.0,theater,0.0,2.0,NaN,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Tenderloin,94102.0,"(37.7867728692759, -122.41058135889531)",1426927300600
16106,201306189859,8,otc alterations permit,06/18/2013,1672,032,820,NaN,30th,Av,NaN,NaN,repair existing stairs in kind for dry rot wat...,issued,06/20/2013,06/18/2013,06/20/2013,NaN,06/20/2013,NaN,3.0,3.0,NaN,NaN,06/15/2014,4200.0,11000.0,apartments,4.0,apartments,4.0,2.0,NaN,5.0,wood frame (5),5.0,wood frame (5),NaN,1.0,Outer Richmond,94121.0,"(37.77379233382766, -122.48932155802034)",1308459268942
18985,201307182170,8,otc alterations permit,07/18/2013,0174,008,710,NaN,Sansome,St,NaN,NaN,basement/1st/2nd/3rd floors-t.i to include new...,issued,08/01/2013,07/18/2013,08/01/2013,NaN,08/01/2013,NaN,3.0,3.0,NaN,NaN,07/16/2016,450000.0,499000.0,office,0.0,office,0.0,2.0,NaN,3.0,constr type 3,3.0,constr type 3,NaN,3.0,Chinatown,94111.0,"(37.797049842608764, -122.40158254347386)",131147961563
156440,M783867,8,otc alterations permit,05/09/2017,2654,047,300,NaN,Corbett,Av,0.0,NaN,street space,issued,05/09/2017,05/09/2017,05/09/2017,NaN,05/09/2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,Castro/Upper Market,94114.0,"(37.76052555262265, -122.44456676148731)",1462346459971
30727,201311121601,8,otc alterations permit,11/12/2013,3514,030,1663,NaN,Mission,St,NaN,NaN,7th floor: add 4 sprinkler heads. move one spr...,complete,12/10/2013,11/12/2013,11/12/2013,12/10/2013,11/12/2013,NaN,7.0,7.0,NaN,Y,11/07/2014,2500.0,2500.0,office,0.0,office,0.0,2.0,NaN,1.0,constr type 1,1.0,constr type 1,NaN,6.0,Mission,94103.0,"(37.771147157928574, -122.41919768460646)",1323794156496
143153,201611213212,8,otc alterations permit,11/21/2016,5338,042,1747,NaN,Quesada,Av,NaN,NaN,reroofing,complete,12/01/2016,11/21/2016,11/21/2016,12/01/2016,11/21/2016,NaN,2.0,2.0,NaN,NaN,11/16/2017,8000.0,8000.0,2 family dwelling,2.0,2 family dwelling,2.0,0.0,NaN,5.0,wood frame (5),5.0,wood frame (5),NaN,10.0,Bayview Hunters Point,94124.0,"(37.733582791846665, -122.39241108367541)",1445040179979


Depois de ler os dados, descobrimos que ambos os conjuntos de dados têm valores ausentes.

Nosso próximo passo será:
-  calcular o número de valores ausentes que temos em cada coluna.

Para contar os valores nulos, o Pandas têm a função isnull(). Como o dataset principal possui 102 colunas, analisaremos as primeiras dez colunas contendo valores ausentes.

In [14]:
contagem_valores_ausentes = dataset.isnull().sum()
contagem_valores_ausentes[0:20]

,0
Date,0
GameID,0
Drive,0
qtr,0
down,61154
time,224
TimeUnder,0
TimeSecs,224
PlayTimeDiff,444
SideofField,528


Cada nome de coluna e o número associado indicam o número de valores ausentes!

Vamos verificar a porcentagem dos valores ausentes em nosso conjunto de dados.

Para este cálculo de porcentagem, usaremos a combinação das funções prod() do Numpy e shape do Pandas.

In [16]:
# Calculando os totais de celulas e de celulas com valores ausentes (total_nulos)
total_celulas = np.prod(dataset.shape)
total_nulos = contagem_valores_ausentes.sum()
# Calculando o percentual de dados ausentes
print(f"Percentual de Dados ausentes: {(total_nulos/total_celulas):.2%}")

Percentual de Dados ausentes: 27.67%


Agora é sua vez de aplicar as mesmas etapas no conjunto de dados building_permits e verificar a porcentagem de valores ausentes.


In [17]:

contagem_valores_ausentes2 = building_permits.isnull().sum()
contagem_valores_ausentes2[0:20]
contagem_valores_ausentes2 = building_permits.isnull().sum()

In [18]:
# Calculando os totais de celulas e de celulas com valores ausentes (total_nulos)
total_celulas2 = np.prod(building_permits.shape)
total_nulos2 = contagem_valores_ausentes2.sum()
# Calculando o percentual de dados ausentes
print(f"Percentual de Dados ausentes: {(total_nulos2/total_celulas2):.2%}")

Percentual de Dados ausentes: 26.26%


#Eliminar (Drop) valores ausentes.

 - Observação:

Geralmente essa abordagem não é recomendada para projetos importantes! Vale a pena dedicar um tempo para examinar seus dados e realmente examinar todas as colunas com valores ausentes, uma a uma, para realmente conhecer seu conjunto de dados

Se você tem certeza de que deseja descartar linhas com valores ausentes, o pandas tem uma função útil, dropna() para ajudá-lo a fazer isso. Vamos experimentá-lo em nosso conjunto de dados da NFL!

- Atenção:


Se utilizar o comando: dataset.dropna() ele removerá todas as linhas que contém valores ausentes. Neste caso especificamente, resultará em um dataset vazio, pois todas a linhas possuem ao menos um valor ausente. Assim, é mais interessante, indicarmos que sejam apagados apenas as colunas que possuem dados ausentes. Isso é possível, passando um parâmetro para esta função, indicando o eixo: no caso axis="columns".



In [20]:
dataset_semNA = dataset.dropna(axis='columns')
dataset_semNA.head()


(407688, 37)

In [21]:
# Fazendo a comparação entre os datasets
print(f"Colunas no dataset original {dataset.shape[1]}")
print(f"Colunas no dataset sem NA   {dataset_semNA.shape[1]}")

Colunas no dataset original 102
Colunas no dataset sem NA   37


Com essa aplicação perdemos muitas informações do nosso dataset original. Todavia, temos um dataset agora sem valores ausentes!!

#Preenchendo valores ausentes automaticamente
Outra opção é tentar preencher os valores que faltam. Para fins didáticos, estou recortando apenas uma parte do dataset original, para que nosso processo fique mais elucidativo.

In [22]:
# Obtendo uma pequena parte do dataset original
subconjuntoNFL = dataset.loc[:, "EPA":"Season"].head()
subconjuntoNFL

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Podemos usar a função fillna() do Panda para preencher os valores ausentes em um dataframe para nós. Uma opção que temos é especificar o que queremos que os valores NaN sejam substituídos. Vamos fazer a substituição de todos os valores NaN por 0 (zero).

In [23]:
# substituindo todos os valores NaN por 0
subconjuntoNFL.fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


In [24]:
# Preenchendo os valores ausentes com os valores que vem logo a seguir dos NaN

# bfill(axis=0): Este método preenche valores ausentes utilizando o preenchimento para trás (backward fill).
# Isso significa que, para cada célula que tem um valor ausente (NaN), ela será preenchida com o valor da célula
# imediatamente posterior (considerando o eixo 0, ou seja, as linhas).
# Se uma célula não tiver um valor posterior, ela permanecerá como NaN.
# fillna(0): Após a aplicação do bfill, o método fillna é chamado para preencher todos os valores ausentes
# restantes com zero (0). Assim, se ainda houver células com valores ausentes após o preenchimento para trás,
# essas células serão substituídas pelo valor zero.

subconjuntoNFL.bfill(axis=0).fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


In [26]:
# Salvando os dados
subconjuntoNFL.to_csv('/dataset_NFL_limpo1.csv')